In [1]:
import sys

sys.path.append("../")
from library.datasets import ProstateCancerDataset
from library.train import create_dataloader
from entrypoint import prostate_scoring_function
import matplotlib.pyplot as plt
import torch
from library.models import *
import random
import glob
from icecream import ic
import copy
import pprint
from tqdm.auto import tqdm
import pandas as pd

In [2]:
best_prostate_models_combined = [
    i
    for i in glob.glob(
        "../tuning_exp_training_details/pretrained_model_raw_metadata/*Combined*/**/*.pt",
        recursive=True,
    )
]
best_prostate_models_combined.extend([i for i in glob.glob("../mega_tuning_exp_training_details/**/*.pt",
        recursive=True,
    )])

In [3]:
len(best_prostate_models_combined)

548

In [4]:
prostate_dir = "../datasets/train_prostate/train/"
random_seed = 24042

val_dataset = ProstateCancerDataset(
    prostate_dir,
    split_type="val",
    random_seed=random_seed,
    input_slice_count=3,
)
test_dataset = ProstateCancerDataset(
    prostate_dir, split_type="test", random_seed=random_seed, input_slice_count=3
)
test_dataset_avg_fill = ProstateCancerDataset(
    prostate_dir, split_type="test", random_seed=random_seed, input_slice_count=3, avg_fill=True
)
test_dataset_extreme = ProstateCancerDataset(
    prostate_dir, split_type="test_extreme", random_seed=random_seed, input_slice_count=3,
)
test_dataset_extreme_avg_fill = ProstateCancerDataset(
    prostate_dir, split_type="test_extreme", random_seed=random_seed, input_slice_count=3, avg_fill=True
)

Adding raw case data to ChaimeleonDataset:   0%|          | 0/294 [00:00<?, ?it/s]

average image value:   0%|          | 0/294 [00:00<?, ?it/s]

Adding raw case data to ChaimeleonDataset:   0%|          | 0/294 [00:00<?, ?it/s]

average image value:   0%|          | 0/294 [00:00<?, ?it/s]

Adding raw case data to ChaimeleonDataset:   0%|          | 0/294 [00:00<?, ?it/s]

average image value:   0%|          | 0/294 [00:00<?, ?it/s]

Adding raw case data to ChaimeleonDataset:   0%|          | 0/294 [00:00<?, ?it/s]

average image value:   0%|          | 0/294 [00:00<?, ?it/s]

Adding raw case data to ChaimeleonDataset:   0%|          | 0/294 [00:00<?, ?it/s]

average image value:   0%|          | 0/294 [00:00<?, ?it/s]

In [5]:
val_loader = create_dataloader(val_dataset, batch_size=6, shuffle=False)
test_loader = create_dataloader(test_dataset, batch_size=6, shuffle=False)
test_avg_fil_loader = create_dataloader(test_dataset_avg_fill, batch_size=6, shuffle=False)
test_extreme_loader = create_dataloader(test_dataset_extreme, batch_size=6, shuffle=False)
test_extreme_avg_fil_loader = create_dataloader(test_dataset_extreme_avg_fill, batch_size=6, shuffle=False)

In [6]:
model_path_to_base = {
    "ProstateCombinedResnet18PretrainedModel.pt": ProstateCombinedResnet18PretrainedModel(),
    "ProstateCombinedResnet18PretrainedModel_V2_Grid.pt": ProstateCombinedResnet18PretrainedModel_V2_Grid(),
    "ProstateCombinedResnet18PretrainedModel_V2_1_Grid.pt": ProstateCombinedResnet18PretrainedModel_V2_1_Grid(),
}
prostate_model_base = model_path_to_base[
    "_".join(best_prostate_models_combined[0].split("/")[-1].split("_")[3:])
]

In [7]:
# load model
model_state_dict = torch.load(best_prostate_models_combined[0])
model = copy.deepcopy(prostate_model_base)
model.load_state_dict(model_state_dict)
model = model.eval()

In [8]:
# Prostate models loop
epoch_targets = []
epoch_preds = []
epoch_outputs = []
for batch_idx, (images, metadata, targets) in enumerate(val_loader):
    data = images, metadata.squeeze()
    if isinstance(targets, list):
        pfs = targets[1]
        targets = targets[0]
    else:
        targets = targets
    output = model(data)
    pred = output.argmax(dim=1, keepdim=True)

    targets, output, pred = (
        targets.argmax(dim=1, keepdim=True),
        torch.nn.functional.sigmoid(output),
        pred,
    )

    epoch_targets.extend([i.item() for i in targets])
    epoch_preds.extend([i.item() for i in pred])
    epoch_outputs.extend([output[k][1].item() for k, i in enumerate(targets)])

score, score_dict = prostate_scoring_function(epoch_targets, epoch_outputs, epoch_preds)

In [9]:
pprint.pprint(best_prostate_models_combined[0])
pprint.pprint(score_dict)

'../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel_V2_Grid/2023-12-18T00-34-26/fixed_augmentation_less_extreme/[]_frozen/0.0005_starting_learning_rate_0.5_factor/best_val_acc_ProstateCombinedResnet18PretrainedModel_V2_Grid.pt'
{'auc_chai': 0.5416666666666666,
 'balanced_accuracy': 0.5416666666666666,
 'roc_auc': 0.5656565656565657,
 'score': 0.5416666666666667,
 'sensitivity': 0.08333333333333333,
 'sensitivity_recall': 0.08333333333333333,
 'specificity': 1.0}


In [10]:
def load_saved_model(model_path, model_base):
    model_state_dict = torch.load(model_path)
    model = copy.deepcopy(model_base)
    model.load_state_dict(model_state_dict)
    model = model.eval()

    return model

In [11]:
def evaluate_prostate_model(
    eval_dataloader, model_path, model_base, verbose=False, eval_id=None
):
    model = load_saved_model(model_path, model_base)

    epoch_targets = []
    epoch_preds = []
    epoch_outputs = []
    for batch_idx, (images, metadata, targets) in enumerate(eval_dataloader):
        data = images, metadata.squeeze()
        if isinstance(targets, list):
            pfs = targets[1]
            targets = targets[0]
        else:
            targets = targets
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)

        targets, output, pred = (
            targets.argmax(dim=1, keepdim=True),
            torch.nn.functional.sigmoid(output),
            pred,
        )

        epoch_targets.extend([i.item() for i in targets])
        epoch_preds.extend([i.item() for i in pred])
        epoch_outputs.extend([output[k][1].item() for k, i in enumerate(targets)])

    score, score_dict = prostate_scoring_function(
        epoch_targets, epoch_outputs, epoch_preds
    )
    if eval_id is not None:
        score_dict = {f"{eval_id}_{key}": val for key, val in score_dict.items()}

    model_details = model_path.split("/")[2:]

    if verbose:
        pprint.pprint(model_details)
        pprint.pprint(score_dict)
        print("\n\n")

    return score_dict

In [12]:
val_results_prostate = {}
for model_path in tqdm(best_prostate_models_combined):
    model_base_name = "_".join(model_path.split("/")[-1].split("_")[3:])
    try:
        model_base = model_path_to_base[model_base_name]
        results = evaluate_prostate_model(
            val_loader, model_path, model_base, eval_id="val"
        )
        val_results_prostate[model_path] = results
        del results
    except Exception as e:
        print(model_path)
        print(str(e))

  0%|          | 0/548 [00:00<?, ?it/s]

In [13]:
test_results_prostate = {}
for model_path in tqdm(best_prostate_models_combined):
    model_base_name = "_".join(model_path.split("/")[-1].split("_")[3:])
    try:
        model_base = model_path_to_base[model_base_name]
        results = evaluate_prostate_model(
            test_loader, model_path, model_base, eval_id="test"
        )
        test_results_prostate[model_path] = results
        del results
    except Exception as e:
        print(model_path)
        print(str(e))

  0%|          | 0/548 [00:00<?, ?it/s]

In [14]:
test_results_avg_fil_prostate = {}
for model_path in tqdm(best_prostate_models_combined):
    model_base_name = "_".join(model_path.split("/")[-1].split("_")[3:])
    try:
        model_base = model_path_to_base[model_base_name]
        results = evaluate_prostate_model(
            test_avg_fil_loader, model_path, model_base, eval_id="test_avg_fil"
        )
        test_results_avg_fil_prostate[model_path] = results
        del results
    except Exception as e:
        print(model_path)
        print(str(e))

  0%|          | 0/548 [00:00<?, ?it/s]

In [15]:
test_results_extreme_prostate = {}
for model_path in tqdm(best_prostate_models_combined):
    model_base_name = "_".join(model_path.split("/")[-1].split("_")[3:])
    try:
        model_base = model_path_to_base[model_base_name]
        results = evaluate_prostate_model(
            test_extreme_loader, model_path, model_base, eval_id="test_extreme"
        )
        test_results_extreme_prostate[model_path] = results
        del results
    except Exception as e:
        print(model_path)
        print(str(e))

  0%|          | 0/548 [00:00<?, ?it/s]

In [16]:
test_results_extreme_avg_fil_prostate = {}
for model_path in tqdm(best_prostate_models_combined):
    model_base_name = "_".join(model_path.split("/")[-1].split("_")[3:])
    try:
        model_base = model_path_to_base[model_base_name]
        results = evaluate_prostate_model(
            test_extreme_avg_fil_loader, model_path, model_base, eval_id="test_extreme_avg_fil"
        )
        test_results_extreme_avg_fil_prostate[model_path] = results
        del results
    except Exception as e:
        print(model_path)
        print(str(e))

  0%|          | 0/548 [00:00<?, ?it/s]

In [17]:
prostate_val_df = pd.DataFrame.from_dict(val_results_prostate, orient="index")
prostate_test_df = pd.DataFrame.from_dict(test_results_prostate, orient="index")
prostate_test_avg_fil_df = pd.DataFrame.from_dict(test_results_avg_fil_prostate, orient="index")
prostate_test_extreme_df = pd.DataFrame.from_dict(test_results_extreme_prostate, orient="index")
prostate_test_extreme_avg_fil_df = pd.DataFrame.from_dict(test_results_extreme_avg_fil_prostate, orient="index")

In [18]:
prostate_val_df

,val_score,val_roc_auc,val_auc_chai,val_balanced_accuracy,val_sensitivity,val_sensitivity_recall,val_specificity
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel_V2_Grid/2023-12-18T00-34-26/fixed_augmentation_less_extreme/[]_frozen/0.0005_starting_learning_rate_0.5_factor/best_val_acc_ProstateCombinedResnet18PretrainedModel_V2_Grid.pt,0.541667,0.565657,0.541667,0.541667,0.083333,0.083333,1.000000
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel_V2_Grid/2023-12-18T00-34-26/fixed_augmentation_less_extreme/[]_frozen/0.0005_starting_learning_rate_0.5_factor/best_val_score_ProstateCombinedResnet18PretrainedModel_V2_Grid.pt,0.704545,0.747475,0.704545,0.704545,0.833333,0.833333,0.575758
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel_V2_Grid/2023-12-18T00-34-26/fixed_augmentation_less_extreme/[]_frozen/0.01_starting_learning_rate_0.1_factor/best_val_acc_ProstateCombinedResnet18PretrainedModel_V2_Grid.pt,0.583333,0.631313,0.583333,0.583333,0.166667,0.166667,1.000000
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel_V2_Grid/2023-12-18T00-34-26/fixed_augmentation_less_extreme/[]_frozen/0.01_starting_learning_rate_0.1_factor/best_val_score_ProstateCombinedResnet18PretrainedModel_V2_Grid.pt,0.606061,0.681818,0.606061,0.606061,1.000000,1.000000,0.212121
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel_V2_Grid/2023-12-18T00-34-26/fixed_augmentation_less_extreme/[]_frozen/0.003_starting_learning_rate_0.5_factor/best_val_acc_ProstateCombinedResnet18PretrainedModel_V2_Grid.pt,0.719697,0.797980,0.719697,0.719697,0.500000,0.500000,0.939394
...,...,...,...,...,...,...,...
"../mega_tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-18T20-42-39/limited_augmentation_reduced_loss_bias_16_4/['layer2', 'layer3']_frozen/0.05_starting_learning_rate_0.25_factor_60_patience/best_val_acc_ProstateCombinedResnet18PretrainedModel.pt",0.662879,0.640152,0.662879,0.662879,0.416667,0.416667,0.909091
"../mega_tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-18T20-42-39/limited_augmentation_reduced_loss_bias_16_4/['layer2', 'layer3']_frozen/0.01_starting_learning_rate_0.1_factor_60_patience/best_val_score_ProstateCombinedResnet18PretrainedModel.pt",0.674242,0.694444,0.674242,0.674242,0.833333,0.833333,0.515152
"../mega_tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-18T20-42-39/limited_augmentation_reduced_loss_bias_16_4/['layer2', 'layer3']_frozen/0.01_starting_learning_rate_0.1_factor_60_patience/best_val_acc_ProstateCombinedResnet18PretrainedModel.pt",0.568182,0.689394,0.568182,0.568182,0.166667,0.166667,0.969697
"../mega_tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-18T20-42-39/limited_augmentation_reduced_loss_bias_16_4/['layer2', 'layer3']_frozen/0.01_starting_learning_rate_0.5_factor_30_patience/best_val_score_ProstateCombinedResnet18PretrainedModel.pt",0.715909,0.714646,0.715909,0.715909,0.583333,0.583333,0.848485


In [19]:
prostate_test_df

,test_score,test_roc_auc,test_auc_chai,test_balanced_accuracy,test_sensitivity,test_sensitivity_recall,test_specificity
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel_V2_Grid/2023-12-18T00-34-26/fixed_augmentation_less_extreme/[]_frozen/0.0005_starting_learning_rate_0.5_factor/best_val_acc_ProstateCombinedResnet18PretrainedModel_V2_Grid.pt,0.500000,0.568182,0.500000,0.500000,0.000000,0.000000,1.000000
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel_V2_Grid/2023-12-18T00-34-26/fixed_augmentation_less_extreme/[]_frozen/0.0005_starting_learning_rate_0.5_factor/best_val_score_ProstateCombinedResnet18PretrainedModel_V2_Grid.pt,0.617424,0.689394,0.617424,0.617424,0.750000,0.750000,0.484848
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel_V2_Grid/2023-12-18T00-34-26/fixed_augmentation_less_extreme/[]_frozen/0.01_starting_learning_rate_0.1_factor/best_val_acc_ProstateCombinedResnet18PretrainedModel_V2_Grid.pt,0.583333,0.664141,0.583333,0.583333,0.166667,0.166667,1.000000
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel_V2_Grid/2023-12-18T00-34-26/fixed_augmentation_less_extreme/[]_frozen/0.01_starting_learning_rate_0.1_factor/best_val_score_ProstateCombinedResnet18PretrainedModel_V2_Grid.pt,0.666667,0.651515,0.666667,0.666667,1.000000,1.000000,0.333333
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel_V2_Grid/2023-12-18T00-34-26/fixed_augmentation_less_extreme/[]_frozen/0.003_starting_learning_rate_0.5_factor/best_val_acc_ProstateCombinedResnet18PretrainedModel_V2_Grid.pt,0.678030,0.785354,0.678030,0.678030,0.416667,0.416667,0.939394
...,...,...,...,...,...,...,...
"../mega_tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-18T20-42-39/limited_augmentation_reduced_loss_bias_16_4/['layer2', 'layer3']_frozen/0.05_starting_learning_rate_0.25_factor_60_patience/best_val_acc_ProstateCombinedResnet18PretrainedModel.pt",0.632576,0.672980,0.632576,0.632576,0.416667,0.416667,0.848485
"../mega_tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-18T20-42-39/limited_augmentation_reduced_loss_bias_16_4/['layer2', 'layer3']_frozen/0.01_starting_learning_rate_0.1_factor_60_patience/best_val_score_ProstateCombinedResnet18PretrainedModel.pt",0.670455,0.689394,0.670455,0.670455,0.916667,0.916667,0.424242
"../mega_tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-18T20-42-39/limited_augmentation_reduced_loss_bias_16_4/['layer2', 'layer3']_frozen/0.01_starting_learning_rate_0.1_factor_60_patience/best_val_acc_ProstateCombinedResnet18PretrainedModel.pt",0.553030,0.684343,0.553030,0.553030,0.166667,0.166667,0.939394
"../mega_tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-18T20-42-39/limited_augmentation_reduced_loss_bias_16_4/['layer2', 'layer3']_frozen/0.01_starting_learning_rate_0.5_factor_30_patience/best_val_score_ProstateCombinedResnet18PretrainedModel.pt",0.715909,0.744949,0.715909,0.715909,0.583333,0.583333,0.848485


In [20]:
prostate_results_df = pd.merge(
    prostate_val_df, prostate_test_df, left_index=True, right_index=True
)
prostate_results_df = pd.merge(
    prostate_results_df, prostate_test_avg_fil_df, left_index=True, right_index=True
)
prostate_results_df = pd.merge(
    prostate_results_df, prostate_test_extreme_avg_fil_df, left_index=True, right_index=True
)
prostate_results_df = pd.merge(
    prostate_results_df, prostate_test_extreme_df, left_index=True, right_index=True
)

In [21]:
prostate_results_df.to_csv("./2023_1_7_prostate_eval_results.csv")

In [22]:
prostate_results_df.to_excel(
    "./2023_01_07_prostate_eval_results_mega_eval_results_classification_final.xlsx",
    sheet_name="prostate_eval_results",
)